In [ ]:
# Built-in modules
import subprocess
from pathlib import Path

# Third-party modules
import py3Dmol
from pymol import cmd 
from ipywidgets import interact, IntSlider

# Custom modules
from dock import AutoDockVina

MOL = '../molecules'

In [ ]:
# Environment construction
conda create -n docking python=3.10
conda install -c conda-forge -c schrodinger pymol-bundle
pip install -U numpy vina
pip install meeko
pip install prody
pip install py3Dmol
pip install ipywidgets

## Prepare receptor protein and ligand


In [ ]:
# Import experimental structure
cmd.fetch(f'{MOL}8ibt')

In [ ]:
# Select chain A

cmd.select('receptor', 'chain A and not resn HOH')
cmd.save(f'{MOL}/8ibt.pdb', 'receptor')
# Select chain C (ligand)
cmd.select('ligand', 'chain C')
cmd.save(f'{MOL}/ligand.mol2', 'ligand') # mol and sdf don't work
# PDB -> PDBQT
command = f'mk_prepare_receptor.py -i {MOL}/8ibt.pdb -o {MOL}/8ibt -p'
subprocess.run(command, shell=True, capture_output=False)
command = f'mk_prepare_ligand.py -i {MOL}/ligand.mol2 -o {MOL}/ligand.pdbqt'
subprocess.run(command, shell=True, capture_output=False)

## Choose docking grid

In [ ]:
def read_mol(file_path: str) -> str:

    with open(file_path, "r") as f:
        pdbqt = f.readlines()
    
    # Extract only the lines starting with ATOM or HETATM
    pdb_lines = [line for line in pdbqt if line.startswith(("ATOM", "HETATM", "MODEL", "ENDMDL"))]
    pdb_str = "".join(pdb_lines)

    return pdb_str

def box2pdb(center: list, size: list, spacing = 1) -> str:
    lengths = [s * spacing for s in size]
    mins = [c - l / 2 for c, l in zip(center, lengths)]
    maxs = [c + l / 2 for c, l in zip(center, lengths)]
    corners = [
        (mins[0], mins[1], mins[2]),
        (maxs[0], mins[1], mins[2]),
        (maxs[0], maxs[1], mins[2]),
        (mins[0], maxs[1], mins[2]),
        (mins[0], mins[1], maxs[2]),
        (maxs[0], mins[1], maxs[2]),
        (maxs[0], maxs[1], maxs[2]),
        (mins[0], maxs[1], maxs[2]),
    ]

    box_pdb = ''
    for i, (x, y, z) in enumerate(corners, 1):
        box_pdb += f'ATOM      {i}   Ne BOX X   {i}    {x:8.3f}{y:8.3f}{z:8.3f}  1.00 10.00          Ne\n'

    box_pdb += '''
    CONECT    1    2
    CONECT    1    4
    CONECT    1    5
    CONECT    2    3
    CONECT    2    6
    CONECT    3    4
    CONECT    3    7
    CONECT    4    8
    CONECT    5    6
    CONECT    5    8
    CONECT    6    7
    CONECT    7    8
    '''
    return box_pdb

In [ ]:
# Read pdbqt files
ligand = read_mol(f'{MOL}/ligand.pdbqt')
receptor = read_mol(f'{MOL}/8ibt.pdbqt')

# Visualize complex
view = py3Dmol.view(width=500, height=500)
view.addModel(receptor, 'pdb')
view.setStyle({'cartoon': {'color':'spectrum', 'opacity': 0.9}})
view.addModel(ligand, 'pdb')
view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
view.zoomTo()
view.show()

In [ ]:
# Create docking grid 
center = [76, -29, -59]
size = [20, 10, 12]
box = box2pdb(center, size)

# Visualize complex + grid
view = py3Dmol.view(width=500, height=500)
view.addModel(receptor, 'pdb')
view.setStyle({'cartoon': {'color':'spectrum', 'opacity': 0.7}})
view.addModel(box, 'pdb')
view.addStyle({'elem': 'Ne'}, {'stick': {}})
view.addModel(ligand, 'pdb')
view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
view.zoomTo()
view.show()

In [ ]:
# Run docking
receptor_file = f'{MOL}/8ibt.pdbqt'
ligand_file = f'{MOL}/ligand.pdbqt'
vina = AutoDockVina(receptor_file, ligand_file)
exhaustiveness = 30
num_poses = 20
vina.dock(
    center=center, 
    box_size=size, 
    exhaustiveness=exhaustiveness, 
    n_poses=num_poses, 
    output_name=f'{MOL}/output'
    ) 

In [ ]:
# Report results
vina.report()

In [ ]:
# Create docking grid 
box = box2pdb(center, size)

# Visualize complex + grid + poses
def slider(pose: int):
    print(f'Pose {pose}')
    poses = read_mol(f'{MOL}/output.pdbqt')
    pose = poses.split('ENDMDL\n')[:-1][pose - 1]
    view = py3Dmol.view(width=500, height=500)
    view.addModel(ligand, 'pdb')
    view.addStyle({'model':-1}, {"stick": {'colorscheme': 'greenCarbon'}})
    
    view.addModel(box, 'pdb')
    view.addStyle({'elem': 'Ne'}, {'stick': {}})
    view.zoomTo()
    view.addModel(receptor, 'pdb')
    view.setStyle({'model':-1}, {'cartoon': {'color':'spectrum', 'opacity': 0.9}})
    view.addModel(pose, 'pdb')
    view.addStyle({'model':-1}, {"stick": {'colorscheme': 'pinkCarbon'}})
    
    view.show()
    

interact(slider, pose=IntSlider(min=1, max=num_poses - 1, step=1))